<a href="https://colab.research.google.com/github/vangogh1803/face_rec/blob/main/face_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = "/content/drive/MyDrive/face_proj"


Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Subset, Dataset
import os
import numpy as np
from datasets import load_dataset
from PIL import Image

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)


Using: cpu


In [4]:
!pip install -q datasets


In [5]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json


In [10]:
!kaggle datasets list | head


ref                                                  title                                           size  lastUpdated                 downloadCount  voteCount  usabilityRating  
---------------------------------------------------  ----------------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
neurocipher/heartdisease                             Heart Disease                                   3491  2025-12-11 15:29:14.327000           2114        204  1.0              
ahmeduzaki/wind-and-solar-energy-production-dataset  Wind & Solar Energy Production Dataset        395372  2026-01-02 21:06:22.780000              0         35  1.0              
kundanbedmutha/exam-score-prediction-dataset         Exam Score Prediction Dataset                 325454  2025-11-28 07:29:01.047000           5863        249  1.0              
guriya79/heart-failure-prediction-dataset            Heart Failure Clinical Records Study            4067

In [6]:
!kaggle datasets download -d kenny3s/casia-webface

Dataset URL: https://www.kaggle.com/datasets/kenny3s/casia-webface
License(s): MIT
 99% 1.02G/1.03G [00:04<00:00, 243MB/s]
100% 1.03G/1.03G [00:04<00:00, 224MB/s]


In [7]:
!kaggle datasets download -d jessicali9530/lfw-dataset


Dataset URL: https://www.kaggle.com/datasets/jessicali9530/lfw-dataset
License(s): other
  0% 0.00/112M [00:00<?, ?B/s]
100% 112M/112M [00:00<00:00, 1.31GB/s]


In [12]:
!unzip /content/data/lfw-dataset.zip -d /content/data


unzip:  cannot find or open /content/data/lfw-dataset.zip, /content/data/lfw-dataset.zip.zip or /content/data/lfw-dataset.zip.ZIP.


In [8]:
!unzip /content/casia-webface.zip -d /content/data


Streaming output truncated to the last 5000 lines.
  inflating: /content/data/datasets/4879650/009.jpg  
  inflating: /content/data/datasets/4879650/010.jpg  
  inflating: /content/data/datasets/4879650/011.jpg  
  inflating: /content/data/datasets/4879650/012.jpg  
  inflating: /content/data/datasets/4879650/013.jpg  
  inflating: /content/data/datasets/4879650/018.jpg  
  inflating: /content/data/datasets/4879650/020.jpg  
  inflating: /content/data/datasets/4879650/021.jpg  
  inflating: /content/data/datasets/4879650/022.jpg  
  inflating: /content/data/datasets/4883718/001.jpg  
  inflating: /content/data/datasets/4883718/002.jpg  
  inflating: /content/data/datasets/4883718/003.jpg  
  inflating: /content/data/datasets/4883718/004.jpg  
  inflating: /content/data/datasets/4883718/005.jpg  
  inflating: /content/data/datasets/4883718/007.jpg  
  inflating: /content/data/datasets/4883718/008.jpg  
  inflating: /content/data/datasets/4883718/009.jpg  
  inflating: /content/data/data

In [42]:
!ls /content/data/datasets


0000045  0010956  0330380  0651008  1015262  1520988  2155525  2960015
0000099  0011038  0330687  0651067  1015355  1521007  2155567  2961085
0000100  0011042  0330913  0651086  1015684  1521209  2157442  2961993
0000102  0011348  0331516  0651159  1016334  1522635  2157624  2965079
0000103  0011882  0331577  0651441  1016428  1523478  2158763  2965271
0000105  0012078  0331772  0651451  1017334  1523899  2158772  2965415
0000107  0012081  0331906  0652089  1017524  1524066  2158803  2965656
0000108  0012523  0333410  0652445  1017719  1524098  2159829  2966102
0000114  0013037  0333589  0652514  1017951  1524440  2160056  2966255
0000117  0013065  0333804  0652619  1017958  1525011  2160896  2966488
0000119  0013712  0333911  0652663  1018007  1525114  2161450  2966708
0000121  0014582  0333982  0652783  1018488  1525807  2161732  2967091
0000133  0014605  0334106  0653248  1018521  1525948  2161815  2969725
0000137  0014960  0334159  0653293  1018581  1526223  2162024  2970551
000014

# *Classification*

In [9]:
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.Grayscale(num_output_channels=3),
#     transforms.ToTensor(),
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225]
#     )
# ])
##augmentation
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([
        transforms.ColorJitter(
            brightness=0.2,
            contrast=0.2,
            saturation=0.2
        )
    ], p=0.5),
    transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [10]:
!find /content/data -maxdepth 3 -type d | head -n 20



/content/data
/content/data/datasets
/content/data/datasets/0111927
/content/data/datasets/0112001
/content/data/datasets/0112085
/content/data/datasets/0112094
/content/data/datasets/0112379
/content/data/datasets/0112492
/content/data/datasets/0112871
/content/data/datasets/0112932
/content/data/datasets/0113140
/content/data/datasets/0113156
/content/data/datasets/0113158
/content/data/datasets/0113908
/content/data/datasets/0113998
/content/data/datasets/0114175
/content/data/datasets/0114177
/content/data/datasets/0114238
/content/data/datasets/0114256
/content/data/datasets/0114371


In [10]:
dataset_path = "/content/data/datasets"

dataset = datasets.ImageFolder(
    root=dataset_path,
    transform=transform
)

print("Total images:", len(dataset))
print("Total identities:", len(dataset.classes))


Total images: 452960
Total identities: 10575


In [11]:
DEV_IDENTITIES = 350
IMAGES_PER_ID = 40
import os, shutil
from collections import defaultdict

SRC = "/content/data/datasets"
DST = "/content/data/casia_dev"

os.makedirs(DST, exist_ok=True)


In [12]:
identities = sorted(os.listdir(SRC))[:DEV_IDENTITIES]

for ident in identities:
    src_id = os.path.join(SRC, ident)
    dst_id = os.path.join(DST, ident)
    os.makedirs(dst_id, exist_ok=True)

    imgs = os.listdir(src_id)[:IMAGES_PER_ID]
    for img in imgs:
        shutil.copy(
            os.path.join(src_id, img),
            os.path.join(dst_id, img)
        )

print("CASIA dev subset ready")


CASIA dev subset ready


In [13]:
dataset_path = "/content/data/casia_dev"

dataset = datasets.ImageFolder(
    root=dataset_path,
    transform=transform
)

print("Images:", len(dataset))
print("Identities:", len(dataset.classes))


Images: 12263
Identities: 350


In [14]:
train_size=int(0.8*len(dataset))
val_size=len(dataset)-train_size
train_dataset, val_dataset=torch.utils.data.random_split(
    dataset, [train_size, val_size]
  )
train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader=DataLoader(val_dataset, batch_size=32, shuffle=False)

In [15]:
from torch.utils.data import Subset
import random

labels = dataset.targets
id_to_idx = {}

for i, lbl in enumerate(labels):
    id_to_idx.setdefault(lbl, []).append(i)

ids = list(id_to_idx.keys())
random.shuffle(ids)

split = int(0.8 * len(ids))
train_ids = ids[:split]
val_ids = ids[split:]

train_idx, val_idx = [], []

for i in train_ids:
    train_idx.extend(id_to_idx[i])

for i in val_ids:
    val_idx.extend(id_to_idx[i])

train_dataset = Subset(dataset, train_idx)
val_dataset = Subset(dataset, val_idx)


In [16]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)


In [17]:
model=models.resnet18(pretrained=True)
model.fc=nn.Linear(model.fc.in_features, len(dataset.classes))
model=model.to(device)

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 60.7MB/s]


In [18]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=1e-4)

In [19]:
#train now:
Epochs=3
for epoch in range(Epochs):
  model.train()
  running_loss=0
  correct=0
  total=0

  for images, labels in train_loader:
    images=images.to(device)
    labels=labels.to(device)

    optimizer.zero_grad()
    output=model(images)
    loss=criterion(output, labels)

    loss.backward()
    optimizer.step()

    running_loss+=loss.item()
    _, preds=torch.max(output, 1)
    correct+=(preds==labels).sum().item()
    total+=labels.size(0)

    train_acc=100*correct/total
    print(f"Epoch [{epoch+1}/{Epochs}] "
          f"Loss: {running_loss:.3f} | Train Acc: {train_acc:.2f}%")


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch [1/3] Loss: 5.991 | Train Acc: 0.00%
Epoch [1/3] Loss: 12.092 | Train Acc: 1.56%
Epoch [1/3] Loss: 18.316 | Train Acc: 1.04%
Epoch [1/3] Loss: 24.427 | Train Acc: 0.78%
Epoch [1/3] Loss: 30.398 | Train Acc: 0.62%
Epoch [1/3] Loss: 36.444 | Train Acc: 0.52%
Epoch [1/3] Loss: 42.553 | Train Acc: 0.45%
Epoch [1/3] Loss: 48.429 | Train Acc: 0.78%
Epoch [1/3] Loss: 54.234 | Train Acc: 0.69%
Epoch [1/3] Loss: 60.295 | Train Acc: 0.94%
Epoch [1/3] Loss: 66.175 | Train Acc: 0.85%
Epoch [1/3] Loss: 72.260 | Train Acc: 0.78%
Epoch [1/3] Loss: 78.098 | Train Acc: 0.72%
Epoch [1/3] Loss: 84.006 | Train Acc: 0.67%
Epoch [1/3] Loss: 89.882 | Train Acc: 0.62%
Epoch [1/3] Loss: 95.902 | Train Acc: 0.59%
Epoch [1/3] Loss: 101.865 | Train Acc: 0.55%
Epoch [1/3] Loss: 107.735 | Train Acc: 0.52%
Epoch [1/3] Loss: 113.750 | Train Acc: 0.49%
Epoch [1/3] Loss: 119.586 | Train Acc: 0.47%
Epoch [1/3] Loss: 125.520 | Train Acc: 0.45%
Epoch [1/3] Loss: 131.361 | Train Acc: 0.43%


KeyboardInterrupt: 

# **image to embeddings**

# ResNet

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        output = model(images)
        _, preds = torch.max(output, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_acc = 100 * correct / total
print("Validation Accuracy:", val_acc)


In [145]:
torch.save(
    model.state_dict(),
    "/content/drive/MyDrive/face_proj/phase1_lfw_resnet18.pth"
)
print("Model saved")


Model saved


In [191]:
print(dataset.classes[:10])


['0000045', '0000099', '0000100', '0000102', '0000103', '0000105', '0000107', '0000108', '0000114', '0000117']


In [192]:
import torch.nn as nn
from torchvision import models

base_model = models.resnet18(pretrained=True)

# Remove classifier entirely
base_model.fc = nn.Identity()

base_model = base_model.to(device)
base_model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [193]:
embeddings = []
labels = []

with torch.no_grad():
    for images, lbls in val_loader:
        images = images.to(device)
        feats = base_model(images)   # [B, 512]

        embeddings.append(feats.cpu())
        labels.append(lbls)

embeddings = torch.cat(embeddings).numpy()
labels = torch.cat(labels).numpy()

print("Embeddings shape:", embeddings.shape)


Embeddings shape: (1672, 512)


In [194]:
from collections import defaultdict
import random
import numpy as np

# Map identity → embedding indices
id_to_indices = defaultdict(list)
for idx, lbl in enumerate(labels):
    id_to_indices[lbl].append(idx)

pairs = []
pair_labels = []

# Positive pairs (same identity)
for lbl, idxs in id_to_indices.items():
    if len(idxs) >= 2:
        i1, i2 = random.sample(idxs, 2)
        pairs.append((i1, i2))
        pair_labels.append(1)

# Negative pairs (different identities)
all_labels = list(id_to_indices.keys())

for _ in range(len(pairs)):
    l1, l2 = random.sample(all_labels, 2)
    i1 = random.choice(id_to_indices[l1])
    i2 = random.choice(id_to_indices[l2])
    pairs.append((i1, i2))
    pair_labels.append(0)

pairs = np.array(pairs)
pair_labels = np.array(pair_labels)

print("Total pairs:", len(pairs))


Total pairs: 120


In [195]:
from sklearn.metrics.pairwise import cosine_similarity

scores = []

for i1, i2 in pairs:
    sim = cosine_similarity(
        embeddings[i1].reshape(1, -1),
        embeddings[i2].reshape(1, -1)
    )[0][0]
    scores.append(sim)

scores = np.array(scores)


In [196]:
same_scores = scores[pair_labels == 1]
diff_scores = scores[pair_labels == 0]

print("Same-person mean similarity:", same_scores.mean())
print("Different-person mean similarity:", diff_scores.mean())


Same-person mean similarity: 0.77587086
Different-person mean similarity: 0.7494857


In [197]:
##EER
from sklearn.metrics import roc_curve
import numpy as np

fpr, tpr, thresholds = roc_curve(pair_labels, scores, pos_label=1)
fnr = 1 - tpr

eer_idx = np.nanargmin(np.abs(fpr - fnr))
eer = fpr[eer_idx]

print("EER:", eer) #shows how many decisions are wrong



EER: 0.43333333333333335


# ResNet+ArcFace Loss function

In [37]:
!pip install insightface


In [210]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, x, label):
        cosine = F.linear(F.normalize(x), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        phi = torch.where(cosine > self.th, phi, cosine - self.mm)

        one_hot = torch.zeros(cosine.size(), device=x.device)
        one_hot.scatter_(1, label.view(-1, 1), 1.0)

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output


In [211]:
from torchvision import models

num_classes = len(dataset.classes)

backbone = models.resnet18(pretrained=True)
backbone.fc = nn.Identity()
backbone = backbone.to(device)

arcface = ArcMarginProduct(
    in_features=512,
    out_features=num_classes,
    s=30.0,
    m=0.5
).to(device)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [212]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(
    list(backbone.parameters()) + list(arcface.parameters()),
    lr=1e-4
)


In [ ]:
for epoch in range(15):   # 3–5 epochs is enough for now
    backbone.train()
    arcface.train()

    total_loss = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        embeddings = backbone(images)
        logits = arcface(embeddings, labels)
        loss = criterion(logits, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch [{epoch+1}] Loss: {total_loss:.3f}")


Epoch [1] Loss: 25.180
Epoch [2] Loss: 25.159
Epoch [3] Loss: 31.175
Epoch [4] Loss: 27.120
Epoch [5] Loss: 28.789
Epoch [6] Loss: 20.977
Epoch [7] Loss: 19.251
Epoch [8] Loss: 15.341
Epoch [9] Loss: 16.065
Epoch [10] Loss: 25.227
Epoch [11] Loss: 34.067
Epoch [12] Loss: 43.947


In [227]:
backbone.eval()

embeddings = []
labels = []

with torch.no_grad():
    for images, lbls in val_loader:
        images = images.to(device)
        feats = backbone(images)
        embeddings.append(feats.cpu())
        labels.append(lbls)

embeddings = torch.cat(embeddings).numpy()
labels = torch.cat(labels).numpy()

from sklearn.preprocessing import normalize
embeddings = normalize(embeddings)


In [228]:
from collections import defaultdict
import random

id_to_indices = defaultdict(list)
for idx, lbl in enumerate(labels):
    id_to_indices[lbl].append(idx)

pairs = []
pair_labels = []   # 1 = same person, 0 = different

# Positive (same identity)
for lbl, idxs in id_to_indices.items():
    if len(idxs) >= 2:
        i1, i2 = random.sample(idxs, 2)
        pairs.append((i1, i2))
        pair_labels.append(1)

# Negative (different identity)
all_labels = list(id_to_indices.keys())

for _ in range(len(pairs)):
    l1, l2 = random.sample(all_labels, 2)
    i1 = random.choice(id_to_indices[l1])
    i2 = random.choice(id_to_indices[l2])
    pairs.append((i1, i2))
    pair_labels.append(0)

pairs = np.array(pairs)
pair_labels = np.array(pair_labels)

print("Total verification pairs:", len(pairs))


Total verification pairs: 120


In [229]:
from sklearn.metrics.pairwise import cosine_similarity

scores = []

for i1, i2 in pairs:
    sim = cosine_similarity(
        embeddings[i1].reshape(1, -1),
        embeddings[i2].reshape(1, -1)
    )[0][0]
    scores.append(sim)

scores = np.array(scores)


In [230]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(pair_labels, scores, pos_label=1)
fnr = 1 - tpr


In [231]:
import numpy as np

eer_index = np.nanargmin(np.abs(fpr - fnr))
eer = fpr[eer_index]

print("EER:", eer)


EER: 0.45


In [74]:
!mkdir -p /content/drive/MyDrive/face_proj
